In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from src.commons import methods
import statistics

from src.rl_environments.env1 import Env1 as Env

import os

## Config

In [2]:
DEFAULT_CONFIG = {
    'alpha_range': [ i / 10 for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}

TEST_CONFIG = {
    'alpha_range': [ (3 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}



In [3]:
PPO_MODEL_NAME = 'PPO_Env3'

## Train PQN

In [4]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

if True:

    n_cpus = os.cpu_count()

    print(f"Number of processors: {n_cpus}")
    env = make_vec_env(lambda: Env(config=DEFAULT_CONFIG), n_envs=n_cpus, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy", env, verbose=1, device='cpu')

    print(model.n_envs)

    total_timesteps = 500_000
    print(f"Training model for {total_timesteps} timesteps")
    # Start training
    model.learn(total_timesteps=total_timesteps)

    model.save(methods.file_path(PPO_MODEL_NAME, 'models'))



Number of processors: 8
Using cpu device
8
Training model for 500000 timesteps
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.3      |
|    ep_rew_mean     | -5.89    |
| time/              |          |
|    fps             | 1125     |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.18        |
|    ep_rew_mean          | -5.94       |
| time/                   |             |
|    fps                  | 1063        |
|    iterations           | 2           |
|    time_elapsed         | 30          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.010642055 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38  

## Generate Test Data

In [5]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    # travel_time = sum(intervals[4:]) - np.random.exponential(scale=alpha * beta)
    travel_time = sum(intervals[4:]) - np.random.gamma(shape=4, scale=alpha*beta)
    # travel_time = np.random.gamma(shape=5, scale=alpha*beta)
    # travel_time = max(beta * 5, travel_time)
    # travel_time = sum(intervals[3:]) * np.random.uniform(0, 1)
    travel_time = max(alpha * beta, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals', 'alpha', 'beta'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(TEST_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals, 'alpha': alpha, 'beta': beta}
    df.loc[i] = row



## Test

In [6]:
from stable_baselines3 import PPO
model = PPO.load(methods.file_path(PPO_MODEL_NAME, 'models'))

env = Env(config=TEST_CONFIG)
rewards = {}
u_rl = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    u_rl[i] = env.cur_time + env.travel_time
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")

df['rewards'] = rewards
df['u_rl'] = u_rl
print(f"Average reward: {df['rewards'].mean()}")

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'u_star_hat': -1, 'last_update': -1}}
Episode 0 reward: -20.369260271145293 | cur_time = 23.43795934926904
Episode 1 reward: -21.28459115296299 | cur_time = 26.54851651425016
Episode 2 reward: -28.83800114714802 | cur_time = 16.800466465298022
Episode 3 reward: -22.602311869811373 | cur_time = 27.34950951423785
Episode 4 reward: -22.078926271352074 | cur_time = 31.628722009721567
Episode 5 reward: -9.987944722473756 | cur_time = 33.91953600188687
Episode 6 reward: -24.897697740049402 | cur_time = 23.759274532164998
Episode 7 reward: -34.529110423241804 | cur_time = 14.352617789272541
Episode 8 reward: -7.1231254436135245 | cur_time = 11.448040436894413
Episode 9 reward: -12.489586028171914 | cur_time = 26.212654952421964
Episode 10 reward: -12.868758265934886 | cur_time = 20.369533930354752
Episode 11 reward: -7.6835628

## Add Comparison Metrics

In [7]:

from pandarallel import pandarallel

# Initialize pandarallel with progress bar enabled
pandarallel.initialize(progress_bar=True)
for i in [0, 3, 5]:
    print(f"Optimal reward at n = {i}")
    df[f'u{i}'] = df.apply(lambda row: row['intervals'][i:].sum(), axis=1)
    df[f'u_star{i}'] = df.parallel_apply(lambda row: methods.get_u_star_binary_fast(row['total'] - i, row['alpha'], row['beta'], row['h'], row['c']), axis=1)
    df[f'optimal_rewards{i}'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row[f'u{i}'], row[f'u_star{i}']), axis=1)

df['direct_leave_rewards'] = df.apply(lambda row: -methods.cal_cost(row['c'], row['h'], row['intervals'][3:].sum(), row[f'travel_time']), axis=1)
df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Optimal reward at n = 0


Optimal reward at n = 3


Optimal reward at n = 5


,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,161.680382,27,"[5.851802382495728, 4.443151015567876, 12.3869...",3.7,2.5,-20.369260,185.118342,225.856862,235.285064,-25.000000,203.174966,206.920049,-25.000000,171.974962,188.067328,-25.000000,-20.747292
1,0.5,25,217.098026,35,"[4.227139408157423, 12.138904390907541, 9.3260...",3.4,2.5,-21.284591,243.646542,286.215725,284.284119,-0.965803,260.523612,258.090516,-1.216548,240.585825,240.661524,-25.000000,-21.712793
2,0.5,25,219.028419,36,"[2.9539363723849963, 5.845504352926213, 7.4590...",3.7,2.0,-28.838001,235.828885,293.504887,251.457075,-21.023906,277.246371,228.940739,-24.152816,238.437379,213.958840,-12.239269,-29.108976
3,0.5,25,63.220841,21,"[7.3273532387943385, 9.388232047537576, 4.8761...",3.3,2.0,-22.602312,90.570350,135.774974,122.783237,-6.495868,114.183256,103.160735,-5.511261,98.975267,90.162483,-4.406392,-25.481208
4,0.5,25,78.083402,20,"[6.548426866021152, 14.175716932079364, 3.6056...",3.4,2.0,-22.078926,109.712124,153.869977,120.199163,-16.835407,129.540191,100.002801,-14.768695,112.657835,86.631434,-13.013200,-25.728394


## Visualize Results

In [8]:
import plotly.graph_objects as go

all_rewards = [x for x in df.columns if 'reward' in x]

# Calculate means
means = df[all_rewards + ['h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Rewards, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[all_rewards + ['h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Rewards, H, and C", yaxis_title="Median Value")
fig.show()

In [9]:
df.head(50)

,h,c,travel_time,total,intervals,alpha,beta,rewards,u_rl,u0,u_star0,optimal_rewards0,u3,u_star3,optimal_rewards3,u5,u_star5,optimal_rewards5,direct_leave_rewards
0,0.5,25,161.680382,27,"[5.851802382495728, 4.443151015567876, 12.3869...",3.7,2.5,-20.369260,185.118342,225.856862,235.285064,-25.000000,203.174966,206.920049,-25.000000,171.974962,188.067328,-25.000000,-20.747292
1,0.5,25,217.098026,35,"[4.227139408157423, 12.138904390907541, 9.3260...",3.4,2.5,-21.284591,243.646542,286.215725,284.284119,-0.965803,260.523612,258.090516,-1.216548,240.585825,240.661524,-25.000000,-21.712793
2,0.5,25,219.028419,36,"[2.9539363723849963, 5.845504352926213, 7.4590...",3.7,2.0,-28.838001,235.828885,293.504887,251.457075,-21.023906,277.246371,228.940739,-24.152816,238.437379,213.958840,-12.239269,-29.108976
3,0.5,25,63.220841,21,"[7.3273532387943385, 9.388232047537576, 4.8761...",3.3,2.0,-22.602312,90.570350,135.774974,122.783237,-6.495868,114.183256,103.160735,-5.511261,98.975267,90.162483,-4.406392,-25.481208
4,0.5,25,78.083402,20,"[6.548426866021152, 14.175716932079364, 3.6056...",3.4,2.0,-22.078926,109.712124,153.869977,120.199163,-16.835407,129.540191,100.002801,-14.768695,112.657835,86.631434,-13.013200,-25.728394
5,0.5,25,118.031876,25,"[12.76555293388882, 9.746617915199558, 5.85660...",3.1,2.5,-9.987945,151.951412,171.927301,178.152991,-25.000000,143.558521,154.581606,-25.000000,136.242403,138.932429,-25.000000,-12.763322
6,0.5,25,17.315761,10,"[4.105279370106107, 10.931769549282006, 5.9270...",3.3,2.5,-24.897698,41.075036,90.870431,66.864159,-12.003136,69.906366,43.066557,-13.419904,32.316885,27.718131,-2.299377,-26.295302
7,0.5,25,97.842603,32,"[5.270743722512391, 6.1779680740556095, 2.4409...",3.9,1.5,-34.529110,112.195221,181.253442,171.653622,-4.799910,167.363811,154.190019,-6.586896,155.722605,142.579318,-6.571644,-34.760604
8,0.5,25,30.789604,16,"[2.103302197396326, 6.152688566904818, 2.82275...",3.2,1.0,-7.123125,42.237644,56.483895,40.441784,-8.021055,45.405146,31.548184,-6.928481,36.294447,25.710065,-5.292191,-7.307771
9,0.5,25,47.451807,13,"[3.336930868403515, 4.774810140841348, 17.2553...",3.3,2.5,-12.489586,73.664462,98.643634,91.187185,-3.728225,73.276549,66.864159,-3.206195,55.371842,50.920008,-2.225917,-12.912371
